Sample

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import sys
from pathlib import Path
from pprint import pprint

import numpy as np

sys.path.append("../src")
from model import RegimeSwitchingModel, MarkovRegressionWithPenalty

import data

In [3]:
df_sample = pd.read_csv("../data/input/sample.csv")
y = df_sample.loc[:, ['y']].to_numpy()
X = df_sample.loc[:, ['X']].to_numpy()

Markov Regression (normal)

In [4]:
model_mr = RegimeSwitchingModel(y, k_regimes=2, exog=X)
model_mr.fit(method="powell")
print(model_mr.res.summary())

                         Markov Switching Model Results                         
Dep. Variable:                        y   No. Observations:                  519
Model:             RegimeSwitchingModel   Log Likelihood                1157.520
Date:                  Tue, 07 Feb 2023   AIC                          -2301.041
Time:                          12:07:08   BIC                          -2271.278
Sample:                               0   HQIC                         -2289.381
                                  - 519                                         
Covariance Type:                 approx                                         
                             Regime 0 parameters                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const      -6.739e-05      0.003     -0.020      0.984      -0.007       0.006
x1            -0.0365      0.149    

Markov Regression (optimized by cost func. w/ penalty)

In [5]:
model_mrp = MarkovRegressionWithPenalty(y, X, k_regimes=2)

# initial parameters
trend = np.array([-0.002382009, 0.005596649]) # trends by regimes
beta = np.array([[-0.022700795, -0.003124601]]).T # regression coefficients by regimes
variance = np.array([0.032141246**2, 0.016540224**2]) # variances by regimes
log_variance = np.log(variance) # log-variances by regimes
p1 = 2.469036981 # parameter for p11, which is a component of transition matrix
p2 = 2.672129066 # parameter for p22, which is a component of transition matrix
start_params = params = MarkovRegressionWithPenalty.gather_params(
    trend,
    beta,
    log_variance[::-1],
    p1,
    p2
)

# variables of cost func.
weights = model_mrp._exp_decay_weight(t_half=260) # weights
gamma = 5 # penalty parameter
p11_base = 0.92194249 # base p11
p22_base = 0.935361874 # base p22

cost_kwargs = {
    "gamma": gamma,
    "p11_base": p11_base,
    "p22_base": p22_base,
    "epsilon": 1e-20
}

# fit
res = model_mrp.fit(
    params,
    weights,
    **cost_kwargs
)

pprint(res, sort_dicts=False)

Optimization terminated successfully.
         Current function value: -1172.611794
         Iterations: 5
         Function evaluations: 492
{'trend': array([0.00032581, 0.00133219]),
 'beta': array([[-0.05722789],
       [-0.08886282]]),
 'variance': array([0.02752032, 0.01583071]),
 'p1': 5.131441917710099,
 'p2': 4.380307508770849,
 'transition_matrix': array([[0.99412666, 0.01236666],
       [0.00587334, 0.98763334]]),
 'marginal_observation_probabilities': array([3.39352189e+00, 1.50923847e+01, 1.29855466e+01, 8.81854290e+00,
       9.72069479e+00, 1.21320106e+01, 1.54102146e+01, 3.72154980e+00,
       1.13288074e+01, 1.04166873e+01, 8.70894016e+00, 2.76809779e-01,
       1.09672737e+01, 1.46050561e+01, 1.43021765e+01, 1.39462957e+01,
       1.47340802e+01, 9.98153282e-01, 1.30386393e+01, 1.38241484e+01,
       1.41063377e+01, 7.46663387e+00, 3.71149452e+00, 1.16435501e+01,
       1.05645427e+01, 8.40481895e+00, 1.32380438e+01, 5.13264942e+00,
       7.54839201e+00, 1.42652156e+0